<a href="https://colab.research.google.com/github/hijaek/Colab-for-MDX_B/blob/main/(this_is_it)_of_MDX_Colab_%7C_auto_model_downloader_(14_05_23)_(denoiser%2C_model_params%2C_onnxruntime_fix%2C_torch_2_0_comp_)_%7C_HV%2BKoD%2BCrusty%2Bjarredou_(02_05_23).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font color="Red">MDX Colab with new UVR models</font><br>

It downloads all models automatically. <br>No manual models.py replacing necessary for various models - while changing model, read model parameters section below.

<sup><br>Models provided are from [Kuielab](https://github.com/kuielab/mdx-net-submission/), [UVR](https://github.com/Anjok07/ultimatevocalremovergui/) and [Kim](https://github.com/KimberleyJensen/) <br> (you can support UVR [here](https://www.buymeacoffee.com/uvr5/vip-model-download-instructions) and [here](https://boosty.to/uvr)).</sup></br>
<sup><br>Notebook by [Audio Hacker](https://www.youtube.com/channel/UC0NiSV1jLMH-9E09wiDVFYw/), modified by Audio Separation community.</sup></br> 



In [5]:
initialised = True
from time import sleep
from google.colab import output
from google.colab import drive

import sys
import os
import shutil
import psutil
import glob

ai = 'https://github.com/kae0-0/Colab-for-MDX_B'
ai_version = 'https://raw.githubusercontent.com/kae0-0/Colab-for-MDX_B/main/v'
onnx_list = 'https://raw.githubusercontent.com/kae0-0/Colab-for-MDX_B/main/onnx_list'
#@title Initialise MDX
#@markdown Mounting to drive
mount_to_drive = True #@param {type:"boolean"}
#@markdown Mounting path (used only if mount_to_drive is ticked)
mount_path = '/content/drive/MyDrive' #@param ["snippets:","/content/drive/MyDrive","/content/drive/Shareddrives/<your shared drive name>", "/content/drive/Shareddrives/Shared Drive"]{allow-input: true}
ForceUpdate = False #@param {type:"boolean"}
class h:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout
def get_size(bytes, suffix='B'): # read ram
    global svmem
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f'{bytes:.2f}{unit}{suffix}'
        bytes /= factor
    svmem = psutil.virtual_memory()
def console(t):
    get_ipython().system(t)
def LinUzip(file): # unzip call linux, force replace
    console(f'unzip -o {file}')
#-------------------------------------------------------
def getONNX():
    console(f'wget {onnx_list} -O onnx_list')
    _onnx = open("onnx_list", "r")
    _onnx = _onnx.readlines()
    os.remove('onnx_list')
    for model in _onnx:
        _model = sanitize_filename(os.path.basename(model))
        console(f'wget {model}')
        LinUzip(_model)
        os.remove(_model)
    

def getDemucs(_path):
    #https://dl.fbaipublicfiles.com/demucs/v3.0/demucs_extra-3646af93.th
    root = "https://dl.fbaipublicfiles.com/demucs/v3.0/"
    model = {
        'demucs_extra': '3646af93'
    }
    for models in zip(model.keys(),model.values()):
        console(f'wget {root+models[0]+"-"+models[1]}.th -O {models[0]}.th')
    for _ in glob.glob('*.th'):
        if os.path.isfile(os.path.join(os.getcwd(),_path,_)):
            os.remove(os.path.join(os.getcwd(),_path,_))
        shutil.move(_,_path)
def mount(gdrive=False):
    if gdrive:
        try:
            drive.mount("/content/drive", force_remount=True)
        except:
            drive._mount("/content/drive", force_remount=True)
    else:
        pass
def toPath(path='local'):
    if path == 'local':
        os.chdir('/content')
    elif path == 'gdrive':
        os.chdir(mount_path)

def update():
    with h():
        console(f'wget {ai_version} -O nver')
    f = open('nver', 'r')
    nver = f.read()
    f = open('v', 'r')
    cver = f.read()
    if nver != cver or ForceUpdate:
        print('New update found! {}'.format(nver))
        os.chdir('../')
        print('Updating ai...',end=' ')
        with h():
            console(f'git clone {ai} temp_MDX_Colab')
            console('cp -a temp_MDX_Colab/* MDX_Colab/')
            console('rm -rf temp_MDX_Colab')
        print('done')
        os.chdir('MDX_Colab')
        print('Redownloading models... May require 2-3GB (wait 4,5 minutes. Empty recycle bin afterwards.', end=' ')
        with h():
            getDemucs('model/')
            getONNX()
        print('done')
        output.clear()
        os.remove('v')
        os.rename("nver",'v')
        #os.chdir(f'{os.path.join(mount_path,"MDX_Colab")}')
    else:
        os.remove('nver')
        print('Using latest version.')

def past_installation():
    return os.path.exists('MDX_Colab')

def LoadMDX():
    console(f'git clone {ai} MDX_Colab')

#-------------------------------------------------------
# install requirements
print('Installing dependencies will take 45 seconds...',end=' ')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    svmem = psutil.virtual_memory()
    gpu_runtime = False
    with h():
        console('pip3 install onnxruntime==1.14.1')
else:
    gpu_runtime = True
    with h():
        console('pip3 install onnxruntime-gpu==1.14.1')
with h():
    deps = [
            'pathvalidate',
            'youtube-dl',
            'django'
    ]
    for dep in deps:
        console('pip3 install {}'.format(dep))
# import modules
#console('pip3 install torch==1.13.1')
console('pip3 install soundfile==0.12.1')
console('pip3 install librosa==0.9.1')
from pathvalidate import sanitize_filename
print('done')
if not gpu_runtime:
    print(f'GPU runtime is disabled. You have {get_size(svmem.total)} RAM.\nProcessing will be incredibly slow. 😈')
elif gpu_info.find('Tesla T4') >= 0:
    print('You got a Tesla T4 GPU. (speeds are around  10-25 it/s)')
elif gpu_info.find('Tesla P4') >= 0:
    print('You got a Tesla P4 GPU. (speeds are around  8-22 it/s)')
elif gpu_info.find('Tesla K80') >= 0:
    print('You got a Tesla K80 GPU. (This is the common gpu, speeds are around 2-10 it/s)')
elif gpu_info.find('Tesla P100') >= 0:
    print('You got a Tesla P100 GPU. (This is the Second to the fastest gpu, speeds are around  15-42 it/s)')
else:
    if gpu_runtime:
        print('You got an unknown GPU. Please report the GPU you got!')
        ! nvidia-smi
#console('pip3 install demucs')
#-------------------------------------------------------
# Scripting
mount(mount_to_drive)
toPath('gdrive' if mount_to_drive else 'local')
#check for MDX existence
if not past_installation():
    print('First time installation will take around 3-6 minutes.\nThis requires around 2-3 GB Free Gdrive space.\nPlease try not to interup installation process!!')
    print('Downloading AI...',end=' ')
    with h():
        LoadMDX()
    os.chdir('MDX_Colab')
    print('done')
    
    print('Downloading models...',end=' ')
    with h():
        getDemucs('model/')
        getONNX()
    if os.path.isfile('onnx_list'):
        os.remove('onnx_list')
    print('done')

else:
    os.chdir('MDX_Colab')
    update()

#replace main.py & models.py with the new one (base64 encoded zip file)
%cd /content/drive/MyDrive/MDX_Colab
!echo -n 'UEsDBBQACAgIAIlJp1YAAAAAAAAAAAAAAAAJABIAbW9kZWxzLnB5dXAOAAHveFN0bW9kZWxzLnB53RpLb+M2+u5fQSAHSYns2E4KzAarPWy3gx7SOXSnJ8MQZImSiUiUSlGOp4b/+34k9SD1mDitO2hXg0ko8uP3fpEKyYqcccRzFu5nMcszNVz43yOilvZB6Wcv6YxokAtKUVAiSptZWmXFFzlVNFMp2bG8DGaziGR+iDz0OHuBnw+zLI9w6hcB38OrJd+sGfXjmPtlGKQYZk/WLihL6wl9cJEVsSqD8RqGOd9jZj09wvCQAyxMP8D4FlbPs9ksTGEX+j6nB//zfz7alC5+yqMqxc7TDMET4Rj5PqGE+75d4jR2Ueii1EUweHHRjsJ/EpTeZ1Y1W8TTDsqqwMxu0LtIoHAWLUZnZAdALKoS+zyKQawdRQRUlHP0Kad4NoD8EWBapp9JyTWccc4QQYQiFtAE26nGYLd/ERQFppFtLIkHsP4X/1phykmQDpdrECHaOrIJ9cN9QClOSw8UlFfceH/BDIZ+SX7DHqit5IxE2Fu5qAiiiNDEe7m/XzvuFJF/Bzzcf8pZBpTCabCf8fMv9siyMzPf2lcSG+o21QOLO+p5y6cBPrlHmecyJT0TigNmx25cu4v4cSVx35AWDICvKsH9fev0V5RicrkhLMl+e/VpT5sPIKxeAxbV6eCoRZUIuL0IOBVapt6PoOy9fRwJeYZ5xSgA3LWGAbi+d0pTouMgZ/kUcx8CKrskeUX4QEIMGSwPIheptEqDDGZ4wBJApF5S5iJc5OHeRUN1PMsEmAAyyNKx+sVdJNMxhLoH6XWfF95quX4UGdITietdabKV6JJ8eSN1VLPSjjkoe317W0/XI97bJDkGwA4DukVaVdloOtmaaVdBe0oFwyWNPDdXG5ryt7kEShMukhf9HTtCy4ZNuQ+S5d3KhAr3FX2RCVbhAEnsjp35yjHBXwmN8lcAVbVZpGpfzdnql59imvC91xGFZI0ZySMSKjMueG4rd+rh1rQmCHRvJtgOKESYiYJeDy0ROp1PDm0ty0qt9fD2UUSIht7zoKqrKJEAXZqn2q51G1cNAyM7bkxKF+2R6zHDv0KfErWa/Q1DR2NvVnVNdHWbzsddd9tTbc8dAPUzOIBexEQ0m8lmWOhjwkpgAELs7aQ/XtfFuFfaE7O0r7rSPpJ3e6k5GQcZqwdayRZPbMSssSRslgyFj0o/wrTEX2mWNNg3oHptlbKKM1FkG8pNm9V2mqNtpCxsPWklqrrJtdfueligakKTnVyt2bfKfA12SUd31zO85K81vnybKLw1EaNG3425ggY8UanFc0n9Fu4S6xHTPCG6E94ydIGMRK2/XGAwA8GN7BtF0y7qHgpohAS6QXt58x5q3rKm9xEOMNgZYbl6h4dXV/Pwt92yuqZbfgZ+yiIv8Vv+Ob+qf86/gX/ejrnnfOCe4jHs/fvzyliJoEH6h0pE3xB1zbusRgB/xrvqBMI4Efqxnk/P5/SUnpNTcrbQBCDEswWHe1ETbS0IwZ3lUc5RdTu2dvS0o2eA7EUSh9avIRifZHk5c/mbD6G7TgXgu5fNUzdeQC+Fj7ZlW87dagv9vcZqR+1O3z2x+Wlr0Obsy0TBEb2AX/KAYx9aNT4eU6o3EaC2dVqe708r+L8++yk7PZx9fHp8Wn53XoQvBbcWkAyygNvdFcwl5wYHgILCT/Mw4CSnXr9R7GzenykYodwGewNzhCaKFrJPGp2zY/UOuccQFxx9JCn+lPOPeUWjHxjL2VBFDfafIS/nGRLHiiFu1Nm5PUcJY42c+MSh7rhguNwHBbY385Xbb8e3jgGsNC+xHeszk9FjQ+tutN7w7iLVknta5+6iECITM9mKu/UJEkI3K1J8VP15R9ZTRA8Ev/pB6TMMAX10ejJAc59VHGRYug/uyl1v+wC6kJDvXa2RdY3mVQeULfIk6OAEvHly4V/X221nrQGIbgFIcHWjLU+X+hG8a8CNhhy4KjC48ea4kNxtlluQEv5t5UG73dakDJUo6tkRC4YSV8fH1kVzrQbKy0vt3DB+TFl/RcXhpSqeBpu28FrYWAfwjgvI+5wkVV6V9rTv1B7Wdx8FRK7r1kP36CloOtJmRuxOX9gYQvSOSaNXNUqTvOlDbOBDN3s7gLYfal9RcdFqbbrcfUln1fLSnB02ZKszY+JvGoAegIakt30oc9uAjoK9i+WqT02t3noax5s5ma+2U/tHRX63/jV7Nj2Nhm/gWMpf2jz/GhwgqVLluc21uZgEH1a3N8UiKEExgpGAseCL3AI5xelAGUn2X4Ftwq8scNigrb9/LFom5ELLSRdDwFS3uSE03C1X3tg+yuGm5crVaGzrDrLWmliYzbTcDO+avnTBhhTEaqsvQ4wJ2FVDXbdDI3LHQM30iLCGUUY21sqa2DmuJs1GHfqemsSCUpMkg+MYh7ysfSxIk5wRvs88i9ADZhza10OQVliva7WZNq1hRfFaLx8/uOKO1XG7+ZU+r0IMKlBLBHni6xk07FmQWF0EH3zJWaZkfN1jhm0h7K5sVY/+qeRvpsByyjMESWQYU9OY0m6DXnElYXA6ZCs4/g62/vVnswV2C6qUa2zd1Fg2ne/IZkKaQptbbXUzaOQbU6FbZWoHDgCNWcW17Wqupp2hF7VpSjQTsjMubdV5Nxf7o3fyotGvO8WS46z0rF2UH6zavRo3qZsTHsU+PoKTazIrUmYpgx3WTl7ZSpxmQVAbJo/+w28Xo8cUJZSnf7Tw5JeLUejuVOJZgsAcCFjGAcVT34jHPmrAkzJvuVgul6v6DCMuXUYBn70VwKTiQ0fiPazlBc0H/YJmnD9pGk83kKfMNH4zInso8zNB8/Q/aDYjYZLo72YS9a3+j9vkH39Zk+RXM8m47Ne3ifqjiT8/Tibk+SZWOVzPKuhb2aX+C5b/X8PU1U6ZoKtO5l8xqEsc6yd5M1RRqIeE4sgaYBEt1P8AUEsHCKOrdYKLCAAArSQAAFBLAwQUAAgICAAuSadWAAAAAAAAAAAAAAAABwAQAG1haW4ucHl1cAwAASxgS89tYWluLnB53Ttdc9tGku/6FROprgBIIETKdpzwjr51YufOVY7iWtvZ3WJYKBAYiohAAAsMKNEu/4B7ued7u7+4P2G7ewaDAQFQVPxwVYcqW8B8dPf09OdM84yJdVyyTRZVCWcl5xv44GW5c9kuq1gYpJZg8SbPCtEaGaQRq0rOYsHuYrGG70ysecHKsIhzcXLG3girhBFxesMCdpNkyyBh26CIgyXMtoPipnTYKisYvFUbngqWBwWO9k5WRbZhBSCAPwpzGWzyhMuetNrkOy+Jl3Unj+LVahK5rMyqIuQn9ZysSqNVjLSWrFzVzSIrwnX9QbCwP80l8IhvqrD0YJE8qeG/orZWfyXipKz7gzxPdj5NkYPoVXffcCE7S6Aw56HPVyseirKmIUvT+6JKRbwhSqFJkwpN9TuwCRmkvzNFkPh7pLmE73X/OijXwCM5KA/EehskcRQI3rA0jUX8ifvIojTYKO5GvwfpTeaFWcE9NSUr9Fo+/vntr3VjjQnERFRL7kdq9WGWCn4vjA1SLZsgDW54AVyo8rwAGash3AVFCjuvv8udfgUoRVYGiq+wDA2Tx8lJPdGDNQhe1J/2aXyTwgJOnZMwr9hM7rkX8W0cctuCNss5kV/dzioKpmOLxSvVgQ1eXPrBNogTFF7bYbCXnCk4f2qvDuCu2DqOuJ/lwnamJwwe0pAs56mdlYgprZLEZad3pw7uuGqQQ/HJksgvRQR8BfKAGZ780P1NE/Sr2bpTFLspY/oTn13Mk0i3rOI0SJLdtDWkBbLBf3ISJkFZsncFKFkIDJGzcI2+H4MA+b5d8mTlNNByGK8H5XJenKV+yUWV02BXaZGPUue2yOh5wnWQpsDw2bdPDSywPWhClKJ+M2NWtlpZe2sCXJ5EBYuSWmxLG1HO5qdRUW3KU9iGJVCMf8mA4cs2C4OkPF24DfL6xWlhyIs4Fbb1NgsitHPhmoe3eQZtnudZLk+jmcUsZ4goT2S2FLvhIQmAhr0AvfWRkbaUSWy1DSY6vWRFWcoPYOeg3nbTTV1oi5SxApZ9/qJ7Q/gcd7iPww8xX0NqbKBtiWjlg74UgeUyXMjsJ2A2R6FAVsxAq+A93virGeGgV9kimhbhstRfrVQLvYJdEeC9ZAu99nFlpXeLiCeiPp+Wp7gmsIO2QbfDXrCJ1PXT0y/DW4peTDmM1Fz3tCPa4cVs0mncZ/w8XKAOFsJ7k654wdOQvwdjCTqE1gNNufc7yJitd8CleZ6ABmAzysOFRR2W48Kqsy1YowIk3vrx46uXr+95WKFCvlMdsAvWj+8+dtsXxwhVj6YrHd+4SzdyM3draC26Gh9XgDIxV/0LDYe4gRuHvRaqpdVjHixS3N4e0uD+LqnT1kJ3nW3iexfCDIE+oFx5BQed2jQrM7uVF5J6t4G1ZWlWi21ZzJ4+nYzHTkc7wAVtwFmWvEczdJ8NeFpT4dtLQcTZbMYm7ZnQBcSkuReUIHOgQWlQFMHOniOt8G/Roh6GIqwPDWul6cPlKjMQ39s0pJmndnlknV+NO61aHkt0InG55tE31kNz0XKMGqlHZYlJUeI04vftBYbsAhbdamob+lmvrWmQr6zX9xgeoH5/JlGax4sDqkuC8MB+0RiDf1I/93ax6esiAOG6K2LB7Ub8gSrXgOd9kKLWnTtsytEu/Z+yIT7AhvgPsCF+NBv2RKteQJxuOUSI6JesB+WrX2jx0ZK67ZFUE99DommgeUOkUU4E+RNvNok2x1shI4Vd7xiY78HtIqDoJ5RLKPMkFm3W9k/S29ByJvl8vHAtyTnfusjnE8Ruj0wZdS60TRygZ1BY8ekX2P8PzIn/OHO64ofO1FAo8qV3wZYbbrSAULpIqZVdQlZ0b4NPwD80zmXBsqQWIJ1anAZy9IP/ax2E/9AFORmfn9vRD5cmMdJPKKcOzqqZdAbh0UwF8xBOJTujZ4zsxwir1S7z+FK5pnId5Hw+mjROeQMKBRo3U4pFX426r6v01i8hZ60HUEt5Tv5XD4OwAdU/zYQGB7ErRDnqK4RIHvqWnH3iRfaN1XK+csgLA9eeC64JbAacnOgRS+WepVeez8feswVYuNbL8/rl+8XCkMKS3+ApCI986bqN2Ltj4OSyaVkQJ2qm/tsg1S3OqeFd8GFWAQFF11+Xt3GOhhACjhtuAysVCNcA7OxhlLDqeLcdAPmai2Nck8aL3xRu7208PqDrJDUQXgI1Fw3iCzlYE+WctJFBWExpNMwa1Zj3hpisn+PAhRTQ+dSl6VNAvvDCLN/Zbd0G4okuzdSufVtCXHk7vJVt30GG78HMqieM8yFYRmNhLMRVXCQmzeR7m3r9wZP+dHqPnH4P10tOpJLthwkyiAD3AIZjCUG8K8/U6LiM1cdlLa7Ccn11PvJ1TFBZdAfW8SswoaVB6pM4S0NAr36apWhp0CTHJYywG5yOwy4ODKyX6XTErkH0go2HwkDrL/JAbMowL+KwlRzsCs5lkPtXsGmghNDFpMEqncbLavhO15s15LvmRgDVOEtkflptzDW6e116VQNyZJg+fMgCIaWf4tyWWYMrLRHm60tnRH9lhzOSbfAMx8Yf6VAaPHWcTdnn5bwE97740hNG1P6dBgBhtnl8S651Nm/WKYe59ergcwIW/sHzrdYTJDcZxB/rjTzFAJkDUh8HgnZ2plblnEuHkW1ynpYYm4CG6S9I1vcDALXktvOvFUtFAGj4DUUwOE1WGXSl2ct549vzIrvBU1+Ahj4GD6ttkYkgmeGeEVznfP8Ixumd7pVcgIbKuwY8cjh9V2QhHpLAxiKxp808Op0BlwoCRDj2/JROlGGfMYUekMl5uwNWTgYfA4Emkpm0B8ESWqdmmpqjzopwdidbwUcUeDggociTr8vLq86oG57WDl+ObDzm6OocYXT1I4joqE5NHOlV/kvd1iUShCOvbV0agoCl8M9GC4aWrLTtKxdxOWAEQrKgZg8gdJx2kxwMjZOuOiIyVLqe/cAn7jAbn7s13gDFEBzpPbvAlfZnIjV4HcTRAiEOYPE0vthn5EASoen0wItBfECWouwfG+MeH2RvTZI8+QUGl1lh6w4I48Uu5zPZu0qyQDy5cvBkOcyrLkq6iJBj08y/KYLI7jGU+PhZoT2ieTgJf7qsxwds460WtlKsRENk/9K/svHs1evrX968f81evnr1yzX78T9fXv/H6/fMFsEtTxndF3389c9gwNIMEim24gEYuIHcrA5/1OB+jqhFhj4edWKQjCzyiiq1ryGnc9lnyBDzSlhTNkJeeLABtuPRDaPtfHHAHJ9DGnAxOOvApEF6IDrVAiI5HxPrW8KiiXacM4mgF95wfn40pseu7TBBeod/enN9vFwgoWV8kwYJKk1NNajwlIzLdIT/Lzw8PS3zrMR8ZuJ44FnQhIMFguRHE4mKPx2BtVic9Mu8Si7G6uC2k8L0qzVxm/IZ5JMxF1J+IT2hd8t3pe0ABVPIj4FVEuroIbA6h1XGf9YXHXZp6F+d9H61FWsY28qLnC5rWq66yvHW2Z4M5B747IUNNUL12Uw04oo9h7MHAfKH+7icjQyk6HL3jVl/bBEmKBWdIxr24gXDw5vfUiNSVFGTb4ZNe7GTTiRU9DQUO+UygiHTYkTAZ01WPu6ltzeU+iNxkyR0L3JKm9BpKHCap524SXW2bIN0/33eqjW1AMapoGrDIQUaO/0iIzEQVIhUYJYcDgnVJX2VIrIPiNxxjrCRN6PGwzYuUWVUY1zkYolHEovZh6ICA5hteZEEuQzm1QeMWMcrUd9T0vtQJlSrADtv1gThi7HatlHtAzOfg4Gj9EU3TFxIDfqPSciSpceZsrYJS/ds2GMt2FGWa8Biad2RsmisFc1+9wxnf/Iha9XsBq1Mo/JkCm07fUP3zNOwWfpay1MbHio7ibK7FK8oeWEncXpLhTYIXN+wJ/JNSbquCJq16noUAVhHorHVQwmucV2J7h0cJ0BQ9UaYCU3s5U4AZyQNPA0zvG+fWZVYfWeByK75fRTfcGAmCrPlgYNuDmFXGxRD6+rZ5HLydHx59Wx8OXnyPdXjWFRpZKE5QtCqCmcJgJrpyHzrb1n1oVpyGnVg5lvs1jOzHBF/tuQBCMQsQInLLJgoIGuA73qx0CilBMuzoF0x10ozqv8I8Q5jFePeQyfagcaYo5pkot6W6Z4YytOJz+UXMMWChwJk7Lf0pRB8k9M9iMj0Dvdf5ekXsm5NEZJr1Gh5f5Ovr97a0Ef1R7soaZNydoaIEA9LGcbT9XzgwSrbOz4r8RYOQHhU0xEKH8eova/JneGq6yuVuMSLGLtmp4NH2FFiKKWIhaxIo8PqvSo1GzHOLRpjlig8zFpL6dCFht/RprneYz1GOjZ+H/JcdK5L5rjMhVS+TRCntQ9R5Y5o4ek7lxcWVMDnvVQFj+/wq9Y2yDejyK9rIW1rNCJfg8UZo42FxTGroErEzKLmS3XqRCz3wttc9NU8tJ41T/KZJYugjEIlKg20hmigIhLXGv1iUiAbj0L3y/X1X9nPdNxxEI9ME3CtMfxPIUIpChfY/PcqhsSF3OkgkZWg2dYoM8ksObAblDC6PJZYgnOYUOmuEdd7A9dY0Qz8PA6VKqeTeiorXWOM/HC5VCMLGwNeaxmDy6FjukGC5AkhEvSzuXj1YuENTRZTvVvTZm3AyW4CnLUJ7untwWNKSffPhI0WO0iRvrtEolKDKGUnAyoSgv0BV8N9ARt75P5c14ClJWZLDqYaS5Zpnw8xia6MSbxKk0nLKNseifu9EiY6AEDrruqU8HLTY6/vp+w3kN10y7bRbxYa6hsuGJUc0R0KVSzRFJoxSKi82EPG/fg14vUeQ1DFJbSbeHoONKlrQwzrk+wOUooi2DCsZH5QyCgus9yaILpvfSxR6s5vycUd56kiRqE86zMJdNtOOystQ1GbhpbotMVr2MAgNGQsvLVMjJaGIyVBFZfI7QemggLiuVJbCBoZUN0jKQtYqC4rqDxLObwzfp/zIkZCg2RILPTxfU06pU4N8RNv/OTZ98+/fX713ffPJA9OT0/pbzNX3gywGKR2KcBZUSH+cseAsLBKAhlirLlBJMPz0lhUkYy2s5WSKRl0kmRL40vs8GgQVRezCrw221QlXbNvs0QEN1zeuGjiBlWACm1xq0LLkDG91m+fOk2aPQBEneM1Evu1tqc+Rcxy6WwGjR8ex/eS/fzb78ZDs6jEtXfWk/Hzq0Oz+nF99zCHVDJKXnM7IFVjT8kSxjEQweQeBTAIp7QNFEElsldvsRKkyT9UIVZe3/LFJUbbMIZ2QkOVQzA6IhiQJcrRshZXNjqYC14Zpd81KAwKqJXup1TA2Cr5gfhIV/LUY0zSHAOhGaJySNNE2VduS9vvaoTmNWOGBSu34NiL42bWiB+uA+hevEOcANt4jQFvBjEvuMG7dSDUzRKYHvxlzrpa/zv7x//+z3/VRrbMgEO3RfBpx156bxhdCUP/f7OoStnvVQQqugFzADBYnnz69Am8RGKGs+ruVdXDUY9qki10LePONUetJWVcTT12Q7+cVx/0tWtnzWnS2raj+W3fXJwXHYlu0kYXPQbdpEGXHYnuqo0uewy6qwbd9kh0T9roto9B98TQ6jPMO7MiKHYoUJBVbfFHWrwosqJsdhhrtDUCFFaPQgvzMK0jI2MQkkbvqJz70TAmAGPSwKDC70cDuQIgVw0Q8tGPBvIEgDyRQHTtuh5NDKIDjsbPyPr1/UZawX6jLFpvDkg0rYdqhDC+C4cqrPcMncrFB35X0LGlsvYyXNDdjf6RQfeYrgjQWf4EoMFE/YS/w3uNcoOnGwdhfpFrQgpXOEtZLhVkzFgSbJZRwO7lTz2mrEX3vStB4f/qYMB0CAdMsYpF5LWe/qlTnZnDt9f5HZPx45tZY8APBRP6p0SqhI++TH2TJ8d9OG0NVtWFkPdqkOmX5UyuzYZYZ+w0AyKjfWK0Z0b7ldG+NdqfKPaoPyBA0gG3HF8hXesRvO5j0eA0FRVMFk7LqO1Jr+HQ26JIhG0g1GmFIycx/ngNifF9cq2+j2c3vq90iI6Lffo55ox+gulBgL7y1T2Mkgp52qM2C0+XTt9XIR4Mr6ok2VHUnXABAgVBMGSLdBWEP249df5VnUZ9wPsagj9ln8dT7/Pky+pLqQuwehCzkUEbbqRz8k9QSwcIhTUtTggSAAC+OwAAUEsBAhQAFAAICAgAiUmnVqOrdYKLCAAArSQAAAkAEgAAAAAAAAAAAAAAAAAAAG1vZGVscy5weXVwDgAB73hTdG1vZGVscy5weVBLAQIUABQACAgIAC5Jp1aFNS1OCBIAAL47AAAHABAAAAAAAAAAAAAAANQIAABtYWluLnB5dXAMAAEsYEvPbWFpbi5weVBLBQYAAAAAAgACAI4AAAAhGwAAAAA='   | base64 --decode > tmp.zip
!unzip -o tmp.zip
!rm -rf tmp.zip

################
#outro
print('Success!')

Installing dependencies will take 45 seconds... Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
done
You got a Tesla T4 GPU. (speeds are around  10-25 it/s)
Mounted at /content/drive
Using latest version.
/content
Archive:  tmp.zip
  inflating: models.py               
  inflating: main.py                 
Success!


#<font color="Red">Inference</font>

In [6]:
#@markdown ### Print a list of tracks
for i in glob.glob('tracks/*'):
    print(os.path.basename(i))

정상수 - 킬링버스.wav
정상수 - 블루라군.wav
정상수 - NEW DAY.wav
정상수-반말하지 마라.wav
정상수 - UP & DOWN.wav
정상수 - 명사수.wav
정상수 - 높이뛰기.wav
정상수 - 부산을 대표해.wav
정상수 - 춤추는 가야금.wav
정상수 - 부산그루브.wav
Butterfly.wav


In [14]:
if not 'initialised' in globals():
    raise NameError('Please run the first cell first!! #scrollTo=H_cTbwhVq4K6')
# Modifiable variables
tracks_path = 'tracks/'
separated_path = 'separated/'



# directory = "contents/drive/MyDrive/MDX_Colab/tracks" #@param {type:"string"}

#@markdown ---
#@markdown ## Models to run in order. 
#@markdown ###### In this example, all tracks will go through (Kim vocal 2), then the output will go through (Karokee2) and then (deverb)
#@markdown ###### This is different from ensemble as the tracks are independently run through models in order

ONNX = "MDX-UVR-Kim Vocal Model 2 (Kim vocal 2)" #@param ["off", "(de)Reverb HQ By FoxJoy", "Karokee", "Karokee_AGGR", "Karokee 2", "baseline", "MDX-UVR Ins Model 415", "MDX-UVR Ins Model 418", "MDX-UVR Ins Model 464", "MDX-UVR Ins Model 496 - inst main-MDX 2.1", "Kim ft other instrumental model", "MDX-UVR Vocal Model 406", "MDX-UVR Vocal Model 427", "MDX-UVR-Kim Vocal Model (old)", "MDX-UVR-Kim Vocal Model 2 (Kim vocal 2)", "MDX-UVR Ins Model Full Band 292", "MDX-UVR Ins Model Full Band 403", "MDX-UVR Ins Model Full Band 450 (HQ_1)", "MDX-UVR Ins Model Full Band 498 (HQ_2)"]
ONNX2 = "Karokee 2" #@param ["off", "(de)Reverb HQ By FoxJoy", "Karokee", "Karokee_AGGR", "Karokee 2", "baseline", "MDX-UVR Ins Model 415", "MDX-UVR Ins Model 418", "MDX-UVR Ins Model 464", "MDX-UVR Ins Model 496 - inst main-MDX 2.1", "Kim ft other instrumental model", "MDX-UVR Vocal Model 406", "MDX-UVR Vocal Model 427", "MDX-UVR-Kim Vocal Model (old)", "MDX-UVR-Kim Vocal Model 2 (Kim vocal 2)", "MDX-UVR Ins Model Full Band 292", "MDX-UVR Ins Model Full Band 403", "MDX-UVR Ins Model Full Band 450 (HQ_1)", "MDX-UVR Ins Model Full Band 498 (HQ_2)"]
ONNX3 = "(de)Reverb HQ By FoxJoy" #@param ["off", "(de)Reverb HQ By FoxJoy", "Karokee", "Karokee_AGGR", "Karokee 2", "baseline", "MDX-UVR Ins Model 415", "MDX-UVR Ins Model 418", "MDX-UVR Ins Model 464", "MDX-UVR Ins Model 496 - inst main-MDX 2.1", "Kim ft other instrumental model", "MDX-UVR Vocal Model 406", "MDX-UVR Vocal Model 427", "MDX-UVR-Kim Vocal Model (old)", "MDX-UVR-Kim Vocal Model 2 (Kim vocal 2)", "MDX-UVR Ins Model Full Band 292", "MDX-UVR Ins Model Full Band 403", "MDX-UVR Ins Model Full Band 450 (HQ_1)", "MDX-UVR Ins Model Full Band 498 (HQ_2)"]



## @markdown ### Input track
## @markdown Enter any link/Filename (Upload your songs in tracks folder)
# track = "\u110C\u1165\u11BC\u1109\u1161\u11BC\u1109\u116E - UP & DOWN.wav" #@param {type:"string"}

def uvr(ONNX, Demucs: bool, track):
  #@markdown ---
  #@markdown ### Parameters
  denoise = True #@param {type:"boolean"}
  normalise = True #@param {type:"boolean"}
  amplitude_compensation = "1.033" #@param ["1", "1.02", "1.022099", "1.307", "1.033", "1.035437", "1.03597672895", "1.0568175092136585", "1.065", "1.08"]
  mixing_algorithm = 'default' #@param ["default","min_mag","max_mag"]
  chunks = 45 #@param {type:"slider", min:1, max:55, step:1}
  shifts = 5 #@param {type:"slider", min:0, max:10, step:0.1}
  n_fft = "6144" #@param ["7680", "6144", "5120", "4096"]
  dim_f = "3072" #@param ["2048", "3072"]
  dim_t = "8" #@param ["7", "8", "9"]
  ##validate values
  track = track if 'http' in track else tracks_path+track
  normalise = '--normalise' if normalise else ''
  denoise = '--denoise' if denoise else ''

  if ONNX == 'off':
      pass
  else:
      ONNX = 'onnx/'+ONNX
  if Demucs == False:
      pass
  else:
      Demucs = 'model/'+'demucs_extra.th'
  #@markdown ---
  #@markdown ### Stems
  bass = False #@param {type:"boolean"}
  drums = False #@param {type:"boolean"}
  others = False #@param {type:"boolean"}
  vocals = True #@param {type:"boolean"}
  #@markdown ---
  #@markdown ### Invert stems to mixture
  invert_bass = False #@param {type:"boolean"}
  invert_drums = False #@param {type:"boolean"}
  invert_others = False #@param {type:"boolean"}
  invert_vocals = True #@param {type:"boolean"}
  invert_stems = []
  stems = []
  if bass:
      stems.append('b')
  if drums:
      stems.append('d')
  if others:
      stems.append('o')
  if vocals:
      stems.append('v')

  invert_stems = []
  if invert_bass:
      invert_stems.append('b')
  if invert_drums:
      invert_stems.append('d')
  if invert_others:
      invert_stems.append('o')
  if invert_vocals:
      invert_stems.append('v')

  margin = 44100

  console(f"python main.py --n_fft {n_fft} --dim_f {dim_f} --dim_t {dim_t} --margin {margin} -i \"{track}\" --mixing {mixing_algorithm} --onnx \"{ONNX}\" --model {Demucs} --shifts {round(shifts)} --stems {''.join(stems)} --invert {''.join(invert_stems)} --chunks {chunks} --compensate {amplitude_compensation} {normalise} {denoise}")

# onnx_list = [ONNX, ONNX2, ONNX3]
file_list = [file for file in os.listdir(tracks_path) if os.path.isfile(os.path.join(tracks_path ,file))]

# uvr(ONNX, True, file_list[0]) #킬링버스 485s
for track in file_list:
  print(f'Starting {track}')
  uvr(ONNX, True, track)
  print(f'Finished {track}')

Starting 정상수 - 킬링버스.wav
Loading checkpoint... done
Loading onnx model... done
Processing base: 100% 18/18 [02:49<00:00,  9.41s/it]
 >> done

Processing demucs: 100% 18/18 [03:53<00:00, 12.98s/it]
 >> done

Using ratio: [[0.9]]
--------------------
Inferences finished!
--------------------
Exporting vocals... done
--------------------
Inverting and exporting vocals... done
--------------------
Successfully completed music demixing.
Total time: 496.6s
Finished 정상수 - 킬링버스.wav
Starting 정상수 - 블루라군.wav
Loading checkpoint... done
Loading onnx model... done
Processing base: 100% 5/5 [00:51<00:00, 10.25s/it]
 >> done

Processing demucs: 100% 5/5 [00:49<00:00,  9.99s/it]
 >> done

Using ratio: [[0.9]]
--------------------
Inferences finished!
--------------------
Exporting vocals... done
--------------------
Inverting and exporting vocals... done
--------------------
Successfully completed music demixing.
Total time: 130.0s
Finished 정상수 - 블루라군.wav
Star

In [13]:
find /path/to/source_directory -type f -exec cp {} /path/to/destination_directory \;


'정상수 - 킬링버스.wav'

In [11]:
if not 'initialised' in globals():
    raise NameError('Please run the first cell first!! #scrollTo=H_cTbwhVq4K6')
# Modifiable variables
tracks_path = 'tracks/'
separated_path = 'separated/'

#@markdown ### Input track
#@markdown Enter any link/Filename (Upload your songs in tracks folder)
track = "\u110C\u1165\u11BC\u1109\u1161\u11BC\u1109\u116E - \u110F\u1175\u11AF\u1105\u1175\u11BC\u1107\u1165\u1109\u1173.wav" #@param {type:"string"}

#@markdown ---
#@markdown ### Models
ONNX = "MDX-UVR-Kim Vocal Model 2 (Kim vocal 2)" #@param ["off", "(de)Reverb HQ By FoxJoy", "Karokee", "Karokee_AGGR", "Karokee 2", "baseline", "MDX-UVR Ins Model 415", "MDX-UVR Ins Model 418", "MDX-UVR Ins Model 464", "MDX-UVR Ins Model 496 - inst main-MDX 2.1", "Kim ft other instrumental model", "MDX-UVR Vocal Model 406", "MDX-UVR Vocal Model 427", "MDX-UVR-Kim Vocal Model (old)", "MDX-UVR-Kim Vocal Model 2 (Kim vocal 2)", "MDX-UVR Ins Model Full Band 292", "MDX-UVR Ins Model Full Band 403", "MDX-UVR Ins Model Full Band 450 (HQ_1)", "MDX-UVR Ins Model Full Band 498 (HQ_2)"]
Demucs = 'off'#@param ["off","demucs_extra"]
#@markdown ---
#@markdown ### Parameters
denoise = True #@param {type:"boolean"}
normalise = True #@param {type:"boolean"}
amplitude_compensation = "1.033" #@param ["1", "1.02", "1.022099", "1.307", "1.033", "1.035437", "1.03597672895", "1.0568175092136585", "1.065", "1.08"]
mixing_algorithm = 'min_mag' #@param ["default","min_mag","max_mag"]
chunks = 45 #@param {type:"slider", min:1, max:55, step:1}
shifts = 5.1 #@param {type:"slider", min:0, max:10, step:0.1}
n_fft = "6144" #@param ["7680", "6144", "5120", "4096"]
dim_f = "3072" #@param ["2048", "3072"]
dim_t = "8" #@param ["7", "8", "9"]
##validate values
track = track if 'http' in track else tracks_path+track
normalise = '--normalise' if normalise else ''
denoise = '--denoise' if denoise else ''

if ONNX == 'off':
    pass
else:
    ONNX = 'onnx/'+ONNX
if Demucs == 'off':
    pass
else:
    Demucs = 'model/'+Demucs+'.th'
#@markdown ---
#@markdown ### Stems
bass = False #@param {type:"boolean"}
drums = False #@param {type:"boolean"}
others = False #@param {type:"boolean"}
vocals = True #@param {type:"boolean"}
#@markdown ---
#@markdown ### Invert stems to mixture
invert_bass = False #@param {type:"boolean"}
invert_drums = False #@param {type:"boolean"}
invert_others = False #@param {type:"boolean"}
invert_vocals = True #@param {type:"boolean"}
invert_stems = []
stems = []
if bass:
    stems.append('b')
if drums:
    stems.append('d')
if others:
    stems.append('o')
if vocals:
    stems.append('v')

invert_stems = []
if invert_bass:
    invert_stems.append('b')
if invert_drums:
    invert_stems.append('d')
if invert_others:
    invert_stems.append('o')
if invert_vocals:
    invert_stems.append('v')

margin = 44100

###
# incompatibilities
###
console(f"python main.py --n_fft {n_fft} --dim_f {dim_f} --dim_t {dim_t} --margin {margin} -i \"{track}\" --mixing {mixing_algorithm} --onnx \"{ONNX}\" --model {Demucs} --shifts {round(shifts)} --stems {''.join(stems)} --invert {''.join(invert_stems)} --chunks {chunks} --compensate {amplitude_compensation} {normalise} {denoise}")

Loading onnx model... done
Processing base: 100% 18/18 [02:47<00:00,  9.31s/it]
 >> done

--------------------
Inferences finished!
--------------------
Exporting vocals... done
--------------------
Inverting and exporting vocals... done
--------------------
Successfully completed music demixing.
Total time: 253.6s


Model parameters<br><br>
-*Fullband*: <br>self.n_fft = **6144** dim_f = **3072** dim_t = **8**<br><br>
-*kim vocal 1/2, kim ft other (inst), inst 1-3 (415-464), 406, 427*:<br> self.n_fft = **7680** dim_f = **3072** dim_t = **8**<br><br>
-*496, Karaoke, 9.X (NET-X)*<br>
self.n_fft = **6144** dim_f = **2048** dim_t = **8** (and **9** kuielab_a_vocals only)<br><br>
-Karaoke 2<br>
self.n_fft = **5120** dim_f = **2048** dim_t = **8**
<br><br>
-De-reverb by FoxJoy<br>
self.n_fft = **7680** dim_f = **3072** dim_t = **9**<br><br>
You can use fullband config for e.g. kim ft other, so it will uncap the 17.7kHz cutoff, giving much less muddy output, but it will strongly amplify vocal residues (still, might be useful in some cases).
<br><br>
Volume compensation values<br><br>
-1.02/1.022099 is for Kim other ft (the 1st value recommended)<br>
-1.035 value is for "vocal" models e.g. 427 (and Kim vocal and 9.7 [NET 1]) <br>
-1.033/1.035437 is for HQ_2<br>
-1.05 *compensation* value is for "inst" models (e.g. 464 a.k.a. inst 3) <br>
-1.035/1.307 is for Karaoke models<br>
-1.08 (old) for e.g. 418 model with normalization disabled and chunks 1 (less muddy result for min_mag, sometimes more instruments disappearing, also vs 464)<br>

For instrumental models you can set it to 1 to have the same result like it's in UVR 5 GUI. Amplitude compensation in UVR5 doesn't change instrumental but vocal, but in this Colab it's the opposite due to switched names (and it also never works for inverted stem here). Also, the value above may differ also toward different tracks. Furthemore: 
- Don’t disable *invert_vocals* in Colab even if you only need one stem, otherwise the Colab will end up with error.<br>
- If you won't change any options above, these are the optimized settings for instrumentals with low amount of vocal residues and high quality (for less vocal residues but worse quality, you can also check inst3 and kim ft models).<br>
- Separating same track with different model will be replaced the old result (rename files manually if necessary)
- Inversion applied for second stem allowing getting vocals from instrumental models or vice versa - to get instrumentals from vocal models.
- Output file names for instrumental models will be swapped because the Colab originally was wrtitten for vocal models.<br> 
- In case of memory allocation error (especially with Demucs on), change chunks to 20 or 40 (the more chunks the lower vocal popups and sometimes vocal residues - fixed in UVR5 GUI in batch mode).<br> 
-In case of *system error* with input file name written, retry separation or ensure you wrote your file name correctly.<br>
-In case of *shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory* go to environment and terminate session and start over.<br>
- Downloading result files with Colab file manager can be (very) slow, and multiple file downloading may eventually download only one file - use GDrive instead. 
- Don't add shortcuts to tracks folder. The Colab will not output files properly.
<br>
<br>

**Settings explanation**<br>

*Mixing algorithm* - max_mag - is generally for vocals (gives the most residues in instrumentals), min_mag - for instrumentals (the most aggresive) though "min_mag solve some un-wanted vocal soundings, but instrumental [is] more muffled and less detailed". Check out also "default" as it's in between both - a.k.a. average (it's also required for Demucs enabled which works only for vocal models).<br>

*Chunks* - Set it to 55 or 40 (less aggressive) to alleviate some occasional instrument dissapearing.
Set 1 for the best clarity. It works for at least instrumental model (4:15 track, at least for Tesla T4 (shown at the top) generally better quality, but some instruments tend to disappear more using 1 than 10. For Demucs enabled and/or vocal model it can be set to 10 if your track is below 5:00 minutes. The more chunks, the faster separation up to ~40. For 4:15 track, 72 is max supported till memory allocation error shows up (disabled chunks returns error too). <br>

*Shifts* - can be set max to 10, but it only slightly increases SDR, while processing time is 1.7x longer for each shift and it gives similar result to shifts 5.

*Normalization* - "normalizes all input at first and then changes the wave peak back to original. This makes the separation process better, also less noise" (e.g. if you have to noisy hihats or big amplitude compensation - disable it).
<br>

*Demucs* enabled works correctly with mixing algorithm set to default and only with vocal models (Kim and 427). It's also the only option to get rid of noise of MDX models. Normalization enabled is necessary (but that cnfiguration has slightly more vocal residues than instrumental model). Decrease chunks to 40 if you have ONNXRuntimeError with Demucs on (it requires lower chunks).
<br><br>

*Recommended models*

For instrumentals:
- HQ_2 (best quality; normal vs min_mag less muddy, more vocal residues)

- kim ft other (if you have vocal residues with HQ models, not always better)

- inst 3/464 (if it's still a bit noisy)

- HQ_1 with 1.05 v. comp. (less muddy vs HQ_2 w/ vc. 1.033, but more vocal residues, I like sound of snares in this setting)

For vocals (by SDR):
- kim vocal 2 (less instrumental residues in vocal stem)
- kim vocal 1
<br>or alternatively
- 427 
- 406

For lead vocals:
- Karaokee 2

For backing vocals:
- [HP_KAROKEE-MSB2-3BAND-3090](https://colab.research.google.com/drive/16Q44VBJiIrXOgTINztVDVeb0XKhLKHwl?usp=sharing)

4 Stems: 
- [Demucs ft](https://colab.research.google.com/drive/117SWWC0k9N2MBj7biagHjkRZpmd_ozu1) (don't use baseline and Anjok Vocal here - they're worse and outdated) / [GSEP](https://studio.gaudiolab.io/) / [MDX23](https://mvsep1.ru) (best SDR)


Changelog
<br><br>
Kim's vocal model is "old" only in a sense that she already had a newer one at some point (now added as kim vocal 2), but it was the best evaluated single vocal model by [SDR](https://mvsep.com/quality_checker/leaderboard.php?ensemble=0) available publically for 12.03.23 (now only surpassed by kim vocal 2, and by other dataset, by MDX23 on mvsep1.ru/[Colab](https://github.com/jarredou/MVSep-DMX23-Colab)).

<sup>Added 292 full band beta instrumental model (no 14.7kHz cutoff). 292 epoch already [surpased](https://mvsep.com/quality_checker/leaderboard2.php?&sort=instrum&page=20) Demucs 4 ft model. 309 and 337 epochs are already better than inst 1 SDR-wise.

<sup>Newly added 403 full band beta instrumental model (no 14.7kHz cutoff) - should be better than the 293 in most if not all cases.<br></sup>
Final full band 450 (HQ_1) instrumental model addeded (it may leave more vocal residues than inst3 at times, but it's really worth trying out).<br>
Newer 498 epoch added (HQ_2) "better at removing long drawn out vocals"<br>
Kim other ft model added (the best SDR, even vs 464, but it has cutoff), read point 16.<br>
I've reverted old "Karokee" and "Karokee_AGGR" models, but these are old models (maybe they will do the trick, though), they're already in your MDX_Colab folder after mounting.<br>
Python 3.10 fix by Crusty Crab.<br>
Denoiser and model parameter picker by jarredou. <br>
Karaokee 2 model. Forgot adding it to the list.<br>
Torch 2.0 fix by jarredou. Colab installation speed-up.<br>
Fixed torch fix. <br>
Added Kim vocal 2 - slightly better SDR ("at best it will only perform slightly better or worse on some songs"), (De)reverb HQ (V2, not older Reverb467.onnx) by Fox Joy, also old 406 vocal, and fixed Karaoke 2 for old users.

<sup><br>If you have "vocals.onnx not found" check ForceUpdate or delete MDX_Colab folder from your GDrive, terminate the session and start over.<br>
In case of persistent vocals.onnx error - manual installation of [model files](https://docs.google.com/document/d/17fjNvJzj8ZGSer7c7OFe_CNfUKbAxEh_OBv94ZdRG5c/edit#heading=h.aa2xhwp434) files.